# Scape APIs for options and risk-free rates data

In [79]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import requests
from datetime import datetime as dt
import json

In [80]:
## Options data

## API URL
url_marketData = "https://api.marketdata.app/v1/options/quotes/"

## Option symbol for AAPL Jan 2025 150.000 call
optSymbol = "AAPL250117C00150000/" 

## Parameters for data
end_date = dt.today() # use "atexpiration" for expiration date - doesn't seem to work right now
start_date = end_date - pd.Timedelta(days=5*366) # use "firsttraded" for earliest - doesn't seem to work right now
optParams = {
    "from": start_date, 
    "to": end_date,
    "dateformat": "timestamp",
}

## Get Options data from API
r_option = requests.get(url_marketData + optSymbol, optParams)
data_opt = r_option.json()
print("Options data\n", data_opt.keys(), '\n')

Options data
 dict_keys(['s', 'updated', 'bid', 'bidSize', 'mid', 'ask', 'askSize', 'last', 'openInterest', 'volume', 'inTheMoney', 'intrinsicValue', 'extrinsicValue', 'underlyingPrice']) 



In [81]:
## Data to DataFrame
to_keep = ["updated", "bid", "ask", "underlyingPrice"]
to_drop = ["s", "bidSize", "mid", "askSize", "last", "openInterest", "volume", "inTheMoney", "intrinsicValue", "extrinsicValue"]
df_opt = pd.DataFrame.from_dict(data_opt).drop(columns=to_drop).set_index("updated")
df_opt.index = pd.to_datetime(df_opt.index)
display(df_opt)

## Method to view missing dates (non-trading days)
# print(pd.date_range(start = df_opt.index[0], end = df_opt.index[-1]).difference(df_opt.index))

,bid,ask,underlyingPrice
updated,,,
2022-09-12 16:00:00-04:00,43.00,44.30,163.43
2022-09-13 16:00:00-04:00,37.00,39.30,153.84
2022-09-14 16:00:00-04:00,36.75,39.80,155.31
2022-09-15 16:00:00-04:00,36.65,37.30,152.37
2022-09-16 16:00:00-04:00,35.35,37.75,150.70
...,...,...,...
2023-04-24 16:00:00-04:00,38.00,38.65,165.33
2023-04-25 16:00:00-04:00,36.70,38.00,163.77
2023-04-26 16:00:00-04:00,36.50,37.75,163.76


# Calculate volatilities: implied, historical, beta, VIX

## Historical volatility (HV)
$$HV = \frac{stdev}{\#trading\_days}$$

In [82]:
## Calculate stdev of underlying
underlying_stdev = df_opt['underlyingPrice'].std()

## Calculate time period of data
## check correct time (i.e. is it counted in full days)
time_period = len(df_opt) # Number of trading days

HV = underlying_stdev / time_period

print(HV)

0.06411812164730626


# Calculate risk-free rate of return

In [109]:
## Inflation rate data

## API URL
url_inflation = "https://api.beta.ons.gov.uk/v1/datasets/cpih01"
r_version_data = requests.get(url_inflation + "/editions")

## URL giving links to download datasets in other file formats
url_inflationRate = r_version_data.json()["items"][0]["links"]["latest_version"]["href"]

## Parameter: Aggregate
r_aggregate_options = requests.get(url_inflationRate + "/dimensions/aggregate/options")
# print(json.dumps(r_aggregate_options.json(), indent=2))
aggregate_options = {}
for item in r_aggregate_options.json()["items"]: # iterate through list of dicts
    aggregate_options[item["label"]] = item["option"]
# print(json.dumps(aggregate_options, indent=2))
aggregate_param = aggregate_options["Overall Index"]

## Parameter: Location
r_geography_options = requests.get(url_inflationRate + "/dimensions/geography/options")
# print(json.dumps(r_geography_options.json(), indent=2))
geography_options = {}
for item in r_geography_options.json()["items"]:
    geography_options[item["label"]] = item["option"]
# print(json.dumps(geography_options, indent=2))
geography_param = geography_options["United Kingdom"]

## Parameter: Time
r_time_options = requests.get(url_inflationRate + "/dimensions/time/options")
# print(json.dumps(r_time_options.json(), indent=2))
time_options = []
for item in r_time_options.json()["items"]:
    time_options.append(item["option"])
time_options = sorted(time_options, key=lambda t: dt.strptime(t, "%b-%y"))
# print(time_options)
time_param = time_options[-1]

## Parameters for data
inflParams = { # can use wildcard "*" to retrieve all values for one param at a time
    "time": time_param,
    "geography": geography_param,
    "aggregate": aggregate_param,
}

## Get inflation data from API
r_inflation = requests.get(url_inflationRate + "/observations", inflParams)
data_infl = r_inflation.json()
print("Inflation data")
print(data_infl.keys())
print(json.dumps(data_infl, indent=2))

Inflation data
dict_keys(['@context', 'dimensions', 'limit', 'links', 'observations', 'offset', 'total_observations', 'unit_of_measure'])
{
  "@context": "https://cdn.ons.gov.uk/assets/json-ld/context.json",
  "dimensions": {
    "aggregate": {
      "option": {
        "href": "https://api.beta.ons.gov.uk/v1/code-lists/cpih1dim1aggid/codes/CP00",
        "id": "CP00"
      }
    },
    "geography": {
      "option": {
        "href": "https://api.beta.ons.gov.uk/v1/code-lists/uk-only/codes/K02000001",
        "id": "K02000001"
      }
    },
    "time": {
      "option": {
        "href": "https://api.beta.ons.gov.uk/v1/code-lists/mmm-yy/codes/Mar-23",
        "id": "Mar-23"
      }
    }
  },
  "limit": 10000,
  "links": {
    "dataset_metadata": {
      "href": "https://api.beta.ons.gov.uk/v1/metadata"
    },
    "self": {
      "href": "https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions//observations?aggregate=CP00&geography=K02000001&time=Mar-23"
    },
 

In [120]:
## Data to DataFrame

## Need to collate data differently for multiple dates (from the data_infl["observations"] 
## which is a list -> dict entries for details, observation values correspond to prev dict entry)

obs_time = data_infl["dimensions"]["time"]["option"]["id"]
obs_made = data_infl["total_observations"]
obs_values = data_infl["observations"][0]["observation"]
obs_measure = data_infl["unit_of_measure"]

## WHAT DO I TAKE THE INFLATION BASE YEAR AS? UNIT OF MEASURE? -> EXPECTED LENGTH OF INVESTMENT - CURRENT YEAR?
## Inflation = (Value in Target year - Value in base year) / (Value in Base year) * 100 (%)

column_names = ["Date", "Total observations", "CPIH value", "Unit of measure", "Current Inflation rate"]
df_infl = pd.DataFrame(columns=column_names).set_index("Date")
df_infl.loc[obs_time] = [obs_made, obs_values, obs_measure]
display(df_infl)


,Total observations,CPIH value,Unit of measure
Date,,,
Mar-23,1,126.8,Index: 2015=100


In [ ]:
## Bond yield data
url_bondYields = ""

In [ ]:
# Risk-Free rate of return = 
# current inflation rate - yield of treasury bond (same duration)
# Try BoE API, ONS API, UK Gov API